In [102]:
import urllib.request
import urllib.parse
import xlsxwriter
import os
import time
import sys
import getopt
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re
from urllib.request import urlopen
import parser
import pandas as pd
from collections import defaultdict
import numpy as np

In [201]:

# df_ASSET_TRANSFER = pd.read_csv('C:\\Users\\HS\\Desktop\\3,4차 기업이벤트 양수도 전체 목록(결정 삭제).csv', encoding='ANSI')
df_ASSET_TRANSFER= pd.read_csv('3,4차 기업이벤트 양수도 전체 목록(결정 삭제).csv', encoding = 'ANSI')
df_ASSET_TRANSFER['url'] = df_ASSET_TRANSFER.rcp_no.apply(
    lambda x: "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + str(x))
df_ASSET_TRANSFER['기재정정'] = df_ASSET_TRANSFER['rpt_nm'].isin(
    ['[기재정정]자산양수ㆍ도종료보고서', '[기재정정]합병등종료보고서(자산양수도)', '[기재정정]합병등종료보고서(영업양수도)', '[기재정정]영업양수ㆍ도종료보고서']).apply(int)
rpt_nm_list = [
    ['합병등종료보고서(자산양수도)', '[기재정정]합병등종료보고서(자산양수도)'],  # 8,9
    ['합병등종료보고서(영업양수도)', '[기재정정]합병등종료보고서(영업양수도)'],  # 8,9
    ['영업양수ㆍ도종료보고서', '[기재정정]영업양수ㆍ도종료보고서'],  # 4,5
    ['자산양수ㆍ도종료보고서', '[기재정정]자산양수ㆍ도종료보고서']  # 3,4
]

df_sample = df_ASSET_TRANSFER.loc[df_ASSET_TRANSFER['삭제'].isnull()].loc[
    df_ASSET_TRANSFER.rpt_nm.isin(rpt_nm_list[0])]
# df_sample = df_sample.loc[df_sample.index.isin([0,3,16,17,18])]

In [202]:
df_sample= df_sample.iloc[200:,:]
df_sample

,삭제,bsn_tp,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,url,기재정정
1152,NaN,E003,Y,동양네트웍스,A030790,합병등종료보고서(자산양수도),20150529000132,동양네트웍스,20150529,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
1153,NaN,E003,Y,동양네트웍스,A030790,합병등종료보고서(자산양수도),20150807000230,동양네트웍스,20150807,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
1154,NaN,E003,Y,동양네트웍스,A030790,합병등종료보고서(자산양수도),20180509003116,동양네트웍스,20180509,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
1155,NaN,E003,Y,동양네트웍스,A030790,합병등종료보고서(자산양수도),20180531001491,동양네트웍스,20180531,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
1162,NaN,E003,K,엔에스엔,A031860,합병등종료보고서(자산양수도),20100115000106,엔에스엔,20100115,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
1163,NaN,E003,K,엔에스엔,A031860,합병등종료보고서(자산양수도),20100527000247,엔에스엔,20100527,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
1164,NaN,E003,K,엔에스엔,A031860,합병등종료보고서(자산양수도),20100527000250,엔에스엔,20100527,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
1165,NaN,E003,K,엔에스엔,A031860,합병등종료보고서(자산양수도),20151103000278,엔에스엔,20151103,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
1166,NaN,E003,K,엔에스엔,A031860,합병등종료보고서(자산양수도),20151204000082,엔에스엔,20151204,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
1167,NaN,E003,K,엔에스엔,A031860,합병등종료보고서(자산양수도),20170905000345,엔에스엔,20170905,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0


# 총자산 = 자산총계 처리

In [87]:
dart_div_list = [{'구분': ['자산양수도 전', '증가(감소)', '자산양수도 후'], '자산': ['', '', ''], '유동자산': ['162,257', '+85,350', '247,607'], '비유동자산': ['170,261', '-30,450', '139,811'], '자산총계': ['332,518', '+54,900', '387,418'], '부채': ['', '', ''], '유동부채': ['57,216', '+350', '57,566'], '고정부채': ['18,546', '-', '18,546'], '부채총계': ['75,762', '+350', '76,112'], '자본': ['', '', ''], '자본금': ['27,931', '-', '27,931'], '기타자본항목': ['228,825', '+54,550', '283,375'], '총자본': ['256,756', '+54,550', '311,306'], '자산및부채총계': ['332,518', '+54,900', '387,418'], 'STK_CD': ['A000020'], 'STK_NM_KOR': ['동화약품'], 'rcp_NO': ['20171113000213'], 'rcp_DT': [20171113], 'url': ['http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20171113000213']}, {'유동자산': ['809,433', '41,633', '851,066'], '당좌자산': ['400,638', '361,500', '762,128'], '재고자산': ['8,928', '-', '8,928'], '매각예정비유동자산': ['319,867', '△319,867', '-'], '비유동자산': ['2,209,780', '-', '2,209,780'], '투자자산': ['757,220', '-', '757,220'], '유형자산': ['1,189,865', '-', '1,189,865'], '투자부동산': ['142,900', '-', '142,900'], '무형자산': ['36,139', '-', '36,139'], '기타비유동자산': ['83,656', '-', '83,656'], '자산총계': ['3,019,213', '41,633', '3,060,846'], '유동부채': ['533,272', '-', '533,272'], '비유동부채': ['278,684', '10,075', '288,759'], '부채총계': ['811,956', '10,075', '822,031'], '납입자본과적립금': ['2,207,257', '31,558', '2,238,815'], '자본금': ['114,062', '-', '114,062'], '주식발행초과금': ['2,195,438', '-', '2,195,438'], '이익잉여금': ['359,708', '31,558', '391,266'], '기타포괄손익누계액': ['15,345', '-', '15,345'], '기타자본구성요소': ['(477,296)', '-', '(477,296)'], '자본총계': ['2,207,257', '31,558', '2,238,815'], '매출액': ['1,648,570', '-', '1,648,570'], '영업이익': ['72,133', '-', '72,133'], '계속영업이익': ['35,121', '31,558', '66,679'], '분기순이익': ['35,121', '31,558', '66,679'], '총포괄이익': ['12,709', '31,558', '44,267'], 'STK_CD': ['A000120'], 'STK_NM_KOR': ['CJ대한통운'], 'rcp_NO': ['20120102000266'], 'rcp_DT': [20120102], 'url': ['http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20120102000266']}, {'구분': ['자산양수도 전', '증가(감소)', '자산양수도 후'], '자산': ['', '', ''], '유동자산': ['19,497', '27,047', '46,544'], '비유동자산': ['38,031', '(6)', '38,025'], '자산총계': ['57,528', '27,041', '84,569'], '부채': ['', '', ''], '유동부채': ['21,237', '(6,000)', '15,237'], '비유동부채': ['6,346', '', '6,346'], '부채총계': ['27,583', '(6,000)', '21,583'], '자본': ['', '', ''], '자본금': ['7,146', '', '7,146'], '기타자본항목': ['22,799', '33,041', '55,840'], '자본총계': ['29,945', '33,041', '62,986'], '부채와자본총계': ['57,528', '27,041', '84,569'], 'STK_CD': ['A000650'], 'STK_NM_KOR': ['천일고속'], 'rcp_NO': ['20170920000395'], 'rcp_DT': [20170921], 'url': ['http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170920000395']}, {'자산': ['', '', ''], '유동자산': ['493,947', '493,947', '0'], '비유동자산': ['1,001,001', '1,113,272', '112,271'], '자산총계': ['1,494,948', '1,607,219', '112,271'], '부채': ['', '', '0'], '유동부채': ['617,618', '731,538', '113,920'], '비유동부채': ['238,095', '238,095', '0'], '부채총계': ['855,713', '969,633', '113,920'], '자본': ['', '', '0'], '자본금': ['394,092', '394,092', '0'], '기타불입자본': ['(24,225)', '(24,225)', '0'], '이익잉여금': ['269,782', '268,133', '(1,649)'], '기타자본구성요소': ['(414)', '(414)', '0'], '자본총계': ['639,235', '637,586', '(1,649)'], 'STK_CD': ['A000680'], 'STK_NM_KOR': ['LS네트웍스'], 'rcp_NO': ['20150723000300'], 'rcp_DT': [20150723], 'url': ['http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20150723000300']}, {'과목': ['자산양수·도 전', '증감', '자산양수·도 후'], '자산': ['', '', ''], '유동자산': ['44,237', '5,084', '49,321'], '비유동자산': ['73,658', '(11,250)', '62,408'], '자산총계': ['117,895', '(6,166)', '111,729'], '부채': ['', '', ''], '유동부채': ['47,933', '(24,675)', '23,258'], '비유동부채': ['27,162', '(15,000)', '12,162'], '부채총계': ['75,095', '(39,675)', '35,420'], '자본': ['', '', ''], '납입자본': ['22,820', '-', '22,820'], '이익잉여금': ['12,170', '33,509', '45,679'], '기타본구성요소': ['7,810', '-', '7,810'], '총자본': ['42,800', '33,509', '76,309'], '부채와자본총계': ['117,895', '(6,166)', '111,729'], 'STK_CD': ['A000760'], 'STK_NM_KOR': ['이화산업'], 'rcp_NO': ['20140401001282'], 'rcp_DT': [20140401], 'url': ['http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20140401001282']}]
dart_div_list

[{'구분': ['자산양수도 전', '증가(감소)', '자산양수도 후'],
  '자산': ['', '', ''],
  '유동자산': ['162,257', '+85,350', '247,607'],
  '비유동자산': ['170,261', '-30,450', '139,811'],
  '자산총계': ['332,518', '+54,900', '387,418'],
  '부채': ['', '', ''],
  '유동부채': ['57,216', '+350', '57,566'],
  '고정부채': ['18,546', '-', '18,546'],
  '부채총계': ['75,762', '+350', '76,112'],
  '자본': ['', '', ''],
  '자본금': ['27,931', '-', '27,931'],
  '기타자본항목': ['228,825', '+54,550', '283,375'],
  '총자본': ['256,756', '+54,550', '311,306'],
  '자산및부채총계': ['332,518', '+54,900', '387,418'],
  'STK_CD': ['A000020'],
  'STK_NM_KOR': ['동화약품'],
  'rcp_NO': ['20171113000213'],
  'rcp_DT': [20171113],
  'url': ['http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20171113000213']},
 {'유동자산': ['809,433', '41,633', '851,066'],
  '당좌자산': ['400,638', '361,500', '762,128'],
  '재고자산': ['8,928', '-', '8,928'],
  '매각예정비유동자산': ['319,867', '△319,867', '-'],
  '비유동자산': ['2,209,780', '-', '2,209,780'],
  '투자자산': ['757,220', '-', '757,220'],
  '유형자산': ['1,189,865', '-', '1

In [178]:
sample_dict = {'과목': ['자산양수·도 전', '증감', '자산양수·도 후'],
  '자산': ['', '', ''],
  '유동자산': ['44,237', '5,084', '49,321'],
  '비유동자산': ['73,658', '(11,250)', '62,408'],
  #'자산총계': ['117,895', '(6,166)', '111,729'],
  '부채': ['', '', ''],
  '유동부채': ['47,933', '(24,675)', '23,258'],
  '비유동부채': ['27,162', '(15,000)', '12,162'],
  '부채총계': ['75,095', '(39,675)', '35,420'],
  '자본': ['', '', ''],
  '납입자본': ['22,820', '-', '22,820'],
  '이익잉여금': ['12,170', '33,509', '45,679'],
  '기타본구성요소': ['7,810', '-', '7,810'],
  '총자본': ['42,800', '33,509', '76,309'],
  '부채와자본총계': ['117,895', '(6,166)', '111,729'],
  'STK_CD': ['A000760'],
  'STK_NM_KOR': ['이화산업'],
  'rcp_NO': ['20140401001282'],
  'rcp_DT': [20140401],
  'url': ['http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20140401001282']}

In [179]:
sample_dict=[sample_dict]

In [181]:
import time
start = time.time()

def get_AADJ7(k, sample_dict): 
    kor_name = ['자본총계','총자본','銃자본']
    for kor in kor_name:
        print(kor)
        if kor in sample_dict[k]:
            out = sample_dict[k].get(kor)
            print('kor:',kor)
            print('out:',out)
            break
        else: 
            out = np.nan
    return out


get_AADJ7(0, sample_dict)
print('')
print(time.time() - start)
print(get_AADJ7(0, sample_dict))

자본총계
총자본
kor: 총자본
out: ['42,800', '33,509', '76,309']

0.0
자본총계
총자본
kor: 총자본
out: ['42,800', '33,509', '76,309']
['42,800', '33,509', '76,309']


In [98]:
if '자자' in ['자산총계','총자산','銃자산']:
    print('ho')

In [184]:
cur_dir = os.getcwd()

workbook_name = "DART_dividends_baseline_debugging.xlsx"

workbook = xlsxwriter.Workbook(workbook_name)

# 엑셀 서식지정
worksheet_result = workbook.add_worksheet('result')
filter_format = workbook.add_format({'bold': True,
                                     'fg_color': '#D7E4BC'
                                     })

percent_format = workbook.add_format({'num_format': '0.00%'})

roe_format = workbook.add_format({'bold': True,
                                  'underline': True,
                                  'num_format': '0.00%'})

num_format = workbook.add_format({'num_format': '0.00'})
num2_format = workbook.add_format({'num_format': '#,##0'})
num3_format = workbook.add_format({'num_format': '#,##0.00',
                                   'fg_color': '#FCE4D6'})

worksheet_result.set_column('A:A', 10)
worksheet_result.set_column('B:B', 10)
worksheet_result.set_column('C:C', 10)
worksheet_result.set_column('D:D', 10)
worksheet_result.set_column('H:H', 10)
worksheet_result.set_column('I:I', 10)
worksheet_result.set_column('J:J', 10)
worksheet_result.set_column('K:K', 10)
worksheet_result.set_column('L:L', 10)
worksheet_result.set_column('M:M', 10)
worksheet_result.set_column('N:N', 10)
worksheet_result.set_column('O:O', 10)
worksheet_result.set_column('P:P', 10)

# 엑셀 컬럼명 작성
worksheet_result.write(0, 0, "STK_CD", filter_format)
worksheet_result.write(0, 1, "STK_NM_KOR", filter_format)
worksheet_result.write(0, 2, "rcp_NO", filter_format)
worksheet_result.write(0, 3, "rcp_DT", filter_format)

worksheet_result.write(0, 4, "자산_양수도전_자산총계", filter_format)
worksheet_result.write(0, 5, "자산_증가감소_자산총계", filter_format)
worksheet_result.write(0, 6, "자산_양수도후_자산총계", filter_format)

worksheet_result.write(0, 7, "자산_양수도전_유동자산", filter_format)
worksheet_result.write(0, 8, "자산_증가감소_유동자산", filter_format)
worksheet_result.write(0, 9, "자산_양수도후_유동자산", filter_format)

worksheet_result.write(0, 10, "자산_양수도전_자본총계", filter_format)
worksheet_result.write(0, 11, "자산_증가감소_자본총계", filter_format)
worksheet_result.write(0, 12, "자산_양수도후_자본총계", filter_format)

worksheet_result.write(0, 13, "자산_양수도전_유동부채", filter_format)
worksheet_result.write(0, 14, "자산_증가감소_유동부채", filter_format)
worksheet_result.write(0, 15, "자산_양수도후_유동부채", filter_format)

worksheet_result.write(0, 16, "자산_양수도전_비유동부채", filter_format)
worksheet_result.write(0, 17, "자산_증가감소_비유동부채", filter_format)
worksheet_result.write(0, 18, "자산_양수도후_비유동부채", filter_format)

worksheet_result.write(0, 19, "url", filter_format)

0

In [182]:
def get_AADJ2(k, sample_dict): # 자산총계
    kor_name = ['자산총계','총자산','銃자산']

    for kor in kor_name:
        if kor in sample_dict[k]:
            out = sample_dict[k].get(kor)
            break
        else: 
            out = np.nan

    return out


def get_AADJ8(k, sample_dict): # 유동자산
    kor_name = ['유동자산']

    for kor in kor_name:
        if kor in sample_dict[k]:
            out = sample_dict[k].get(kor)
            break
        else: 
            out = np.nan

    
    return out


def get_AADJ4(k, sample_dict): 
    kor_name = ['자본총계','총자본','銃자본']

    for kor in kor_name:
        if kor in sample_dict[k]:
            out = sample_dict[k].get(kor)
            break
        else: 
            out = np.nan

            
    return out


def get_AADJ6(k, sample_dict): 
    kor_name = ['유동부채']

    for kor in kor_name:
        if kor in sample_dict[k]:
            out = sample_dict[k].get(kor)
            break
        else: 
            out = np.nan
    
    return out


def get_AADJ7(k, sample_dict): 
    kor_name = ['비유동부채', '고정부채']

    for kor in kor_name:
        if kor in sample_dict[k]:
            out = sample_dict[k].get(kor)
            break
        else: 
            out = np.nan
    
    return out

In [185]:
# ver1

start = time.time()


for k in range(len(dart_div_list)):
    
    STK_CD = dart_div_list[k].get('STK_CD')[0]  # 기업코드
    STK_NM_KOR = dart_div_list[k].get('STK_NM_KOR')[0]  #기업명
    rcp_NO = dart_div_list[k].get('rcp_NO')[0]  # 보고서번호
    rcp_DT = dart_div_list[k].get('rcp_DT')[0]  # 보고기일
    url = dart_div_list[k].get('url')[0]  # 보고서url

    asset_before_AADI2 = get_AADJ2(k, dart_div_list)[0]  # 자산_양수도전_자산총계
    asset_increase_AADI2 = get_AADJ2(k, dart_div_list)[1]  # 자산_증가감소_자산총계
    asset_after_AADJ2 = get_AADJ2(k, dart_div_list)[2]  # 자산_양수도후_자산총계

    asset_before_AADI8 = get_AADJ8(k, dart_div_list)[0]  # 자산_양수도전_유동자산
    asset_increase_AADI8 = get_AADJ8(k, dart_div_list)[1]  # 자산_증가감소_유동자산
    asset_after_AADJ8 = get_AADJ8(k, dart_div_list)[2]  # 자산_양수도후_유동자산

    asset_before_AADI4 = get_AADJ4(k, dart_div_list)[0]  # 자산_양수도전_자본총계
    asset_increase_AADI4 = get_AADJ4(k, dart_div_list)[1]  # 자산_증가감소_자본총계
    asset_after_AADJ4 = get_AADJ4(k, dart_div_list)[2]  # 자산_양수도후_자본총계

    asset_before_AADI6 = get_AADJ6(k, dart_div_list)[0]  # 자산_양수도전_유동부채
    asset_increase_AADI6 = get_AADJ6(k, dart_div_list)[1]  # 자산_증가감소_유동부채
    asset_after_AADJ6 = get_AADJ6(k, dart_div_list)[2]  # 자산_양수도후_유동부채

    asset_before_AADI7 = get_AADJ7(k, dart_div_list)[0]  # 자산_양수도전_비유동부채
    asset_increase_AADI7 = get_AADJ7(k, dart_div_list)[1]  # 자산_증가감소_비유동부채
    asset_after_AADJ7 = get_AADJ7(k, dart_div_list)[2]  # 자산_양수도후_비유동부채

    
    worksheet_result.write(k + 1, 0, STK_CD)
    worksheet_result.write(k + 1, 1, STK_NM_KOR)
    worksheet_result.write(k + 1, 2, rcp_DT)
    worksheet_result.write(k + 1, 3, rcp_NO)

    worksheet_result.write(k + 1, 4, asset_before_AADI2)
    worksheet_result.write(k + 1, 5, asset_increase_AADI2)
    worksheet_result.write(k + 1, 6, asset_after_AADJ2)

    worksheet_result.write(k + 1, 7, asset_before_AADI8)
    worksheet_result.write(k + 1, 8, asset_increase_AADI8)
    worksheet_result.write(k + 1, 9, asset_after_AADJ8)

    worksheet_result.write(k + 1, 10, asset_before_AADI4)
    worksheet_result.write(k + 1, 11, asset_increase_AADI4)
    worksheet_result.write(k + 1, 12, asset_after_AADJ4)

    worksheet_result.write(k + 1, 13, asset_before_AADI6)
    worksheet_result.write(k + 1, 14, asset_increase_AADI6)
    worksheet_result.write(k + 1, 15, asset_after_AADJ6)

    worksheet_result.write(k + 1, 16, asset_before_AADI7)
    worksheet_result.write(k + 1, 17, asset_increase_AADI7)
    worksheet_result.write(k + 1, 18, asset_after_AADJ7)

    worksheet_result.write(k + 1, 19, url)

workbook.close()


print('')
print(time.time() - start)




0.05684852600097656


In [105]:
# ver 2

start = time.time()


for k in range(len(dart_div_list)):
    
    STK_CD = dart_div_list[k].get('STK_CD')[0]  # 기업코드
    STK_NM_KOR = dart_div_list[k].get('STK_NM_KOR')[0]  #기업명
    rcp_NO = dart_div_list[k].get('rcp_NO')[0]  # 보고서번호
    rcp_DT = dart_div_list[k].get('rcp_DT')[0]  # 보고기일
    url = dart_div_list[k].get('url')[0]  # 보고서url
    
    
    AADJ2 = get_AADJ2(k, dart_div_list)
    AADJ8 = get_AADJ8(k, dart_div_list)
    AADJ4 = get_AADJ4(k, dart_div_list)
    AADJ6 = get_AADJ6(k, dart_div_list)
    AADJ7 = get_AADJ7(k, dart_div_list)
    

    asset_before_AADI2 = AADJ2[0]  # 자산_양수도전_자산총계
    asset_increase_AADI2 = AADJ2[1]  # 자산_증가감소_자산총계
    asset_after_AADJ2 = AADJ2[2]  # 자산_양수도후_자산총계

    asset_before_AADI8 = AADJ8[0]  # 자산_양수도전_유동자산
    asset_increase_AADI8 = AADJ8[1]  # 자산_증가감소_유동자산
    asset_after_AADJ8 = AADJ8[2]  # 자산_양수도후_유동자산

    asset_before_AADI4 = AADJ4[0]  # 자산_양수도전_자본총계
    asset_increase_AADI4 = AADJ4[1]  # 자산_증가감소_자본총계
    asset_after_AADJ4 = AADJ4[2]  # 자산_양수도후_자본총계

    asset_before_AADI6 = AADJ6[0]  # 자산_양수도전_유동부채
    asset_increase_AADI6 = AADJ6[1]  # 자산_증가감소_유동부채
    asset_after_AADJ6 = AADJ6[2]  # 자산_양수도후_유동부채

    asset_before_AADI7 = AADJ7[0]  # 자산_양수도전_비유동부채
    asset_increase_AADI7 = AADJ7[1]  # 자산_증가감소_비유동부채
    asset_after_AADJ7 = AADJ7[2]  # 자산_양수도후_비유동부채

    
    worksheet_result.write(k + 1, 0, STK_CD)
    worksheet_result.write(k + 1, 1, STK_NM_KOR)
    worksheet_result.write(k + 1, 2, rcp_DT)
    worksheet_result.write(k + 1, 3, rcp_NO)

    worksheet_result.write(k + 1, 4, asset_before_AADI2)
    worksheet_result.write(k + 1, 5, asset_increase_AADI2)
    worksheet_result.write(k + 1, 6, asset_after_AADJ2)

    worksheet_result.write(k + 1, 7, asset_before_AADI8)
    worksheet_result.write(k + 1, 8, asset_increase_AADI8)
    worksheet_result.write(k + 1, 9, asset_after_AADJ8)

    worksheet_result.write(k + 1, 10, asset_before_AADI4)
    worksheet_result.write(k + 1, 11, asset_increase_AADI4)
    worksheet_result.write(k + 1, 12, asset_after_AADJ4)

    worksheet_result.write(k + 1, 13, asset_before_AADI6)
    worksheet_result.write(k + 1, 14, asset_increase_AADI6)
    worksheet_result.write(k + 1, 15, asset_after_AADJ6)

    worksheet_result.write(k + 1, 16, asset_before_AADI7)
    worksheet_result.write(k + 1, 17, asset_increase_AADI7)
    worksheet_result.write(k + 1, 18, asset_after_AADJ7)

    worksheet_result.write(k + 1, 19, url)

workbook.close()


print('')
print(time.time() - start)




0.0009968280792236328


# Table 처리

In [ ]:
# 다른거

현금및현금성자산
지분법적용투자주식
단기차입금
사채
기타자본조정
토지
건축물
기타투자자산


In [211]:

dart_div_list = list()

for i in range(0, 100):
    target = df_sample.iloc[i]
    # print(target)
    crpCd = target.crp_cd
    crpNm = target.crp_nm
    rcpNo = target.rcp_no
    rcpDt = target.rcp_dt
    link = target.url

    handle = urllib.request.urlopen(link)
    print('여기다!!!', handle)
    data = handle.read()
    soup2 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')  # http read
    # print(soup2)

    test = soup2.find('a', {'href': '#download'})['onclick']
    # print(test)

    words = test.split("'")
    # print(words)

    rcpNo = words[1]
    dcmNo = words[3]
    # print(rcpNo)
    # print(dcmNo)

    dart2 = soup2.find_all(string=re.compile('dart2.dtd'))
    dart3 = soup2.find_all(string=re.compile('dart3.xsd'))
    # print(dart3)

    if target['기재정정'] == 0:
        eleId = '8'  # 재무제표 8
    else:
        eleId = '9'  # 재무제표 8

    ####### 뷰어 링크
    if len(dart3) != 0:
        link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=" + eleId + "&offset=4916&length=3668&dtd=dart3.xsd"
    print(link2)

    ####### 테이블
    handle = urllib.request.urlopen(link2)
    # print(handle)
    data = handle.read()
    soup3 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')
    # print(soup3)

    tables = soup3.findAll("table")
    print('len(tables):', len(tables))  # list
    
    
    
    if len(tables) == 0:
        print('해당사항없음, 이후에 이유쓰는 란 추가 ')
        continue
    elif len(tables) == 1:  # 재무제표만
        div_table = soup3.find("table")
    elif 2 <= len(tables) <= 6:  # 단위, 재무제표
        div_table = soup3.findAll("table")[1]
    else: 
        print('일단 한번 보자, 이후에 이유쓰는 란 추가 ')
        break
        
#     print(div_table)  # 예외처리 후 한가지의 table 파싱


    div_trs = div_table.findAll('tr')
    print('len(div_trs)',len(div_trs))
    print(div_trs)

KeyboardInterrupt: 